# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd 
import requests
import xlsxwriter
import math
from scipy import stats

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
from secrets import IEX_CLOUD_API_TOKEN
stocks = pd.read_csv('sp_500_stocks.csv')

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com//stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2354754622177,
 'week52high': 136.9,
 'week52low': 55.9,
 'week52change': 0.790117092473812,
 'sharesOutstanding': 17775642157,
 'float': 0,
 'avg10Volume': 99935012,
 'avg30Volume': 106505169,
 'day200MovingAvg': 114.72,
 'day50MovingAvg': 127.19,
 'employees': 0,
 'ttmEPS': 3.38,
 'ttmDividendRate': 0.8354186517683627,
 'dividendYield': 0.006345068247841052,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-29',
 'nextEarningsDate': '0',
 'peRatio': 39.736973315108195,
 'beta': 1.211301541864665,
 'maxChangePercent': 51.894307653521665,
 'year5ChangePercent': 4.479139903197893,
 'year2ChangePercent': 2.508399042102535,
 'year1ChangePercent': 0.8354666516752962,
 'ytdChangePercent': 0,
 'month6ChangePercent': 0.4620831274860813,
 'month3ChangePercent': 0.1488230120440675,
 'month1ChangePercent': 0.1177673425995632,
 'day30ChangePercent': 0.08242442679860304,
 'day5ChangePercent': 0.005604349707061035}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.8354666516752962

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [17]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch/stats?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(pd.Series([symbol,data[symbol]['price'],data[symbol]['stats']['year1ChangePercent'], 'N/A'], index=my_columns), ignore_index=True)

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,120.91,0.40645,N/A
1,AAL,16.28,-0.465675,N/A
2,AAP,161.93,-0.00905499,N/A
3,AAPL,133.36,0.861114,N/A
4,ABBV,112.15,0.279991,N/A
...,...,...,...,...
500,YUM,113.34,0.103263,N/A
501,ZBH,156.86,0.038256,N/A
502,ZBRA,388.82,0.521831,N/A
503,ZION,43.62,-0.132346,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [18]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,38.09,2.1612,N/A
1,345,NVDA,525.20,1.25211,N/A
2,387,PYPL,238.20,1.18947,N/A
3,275,LB,38.94,1.11128,N/A
4,23,ALB,150.68,1.08076,N/A
5,31,AMD,92.46,1.02294,N/A
6,179,FCX,26.63,0.998526,N/A
7,85,CDNS,138.64,0.980741,N/A
8,339,NOW,564.55,0.963938,N/A
9,236,IDXX,502.41,0.950181,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [21]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter size of your portfolio')

    try:
        float(portfolio_size)
    except ValueError:
        print('Not a number :( \nPlease try again!')
        portfolio_size = input('Enter value of your portfolio')

In [23]:
portfolio_input()

200000.0

In [ ]:
position_size = float(portfolio_size)/len(final_dataframe.index)
position_size

In [25]:
final_dataframe['Number of Shares to Buy'] = (position_size/final_dataframe['Price']).apply(math.floor)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,38.09,2.1612,5250
1,345,NVDA,525.20,1.25211,380
2,387,PYPL,238.20,1.18947,839
3,275,LB,38.94,1.11128,5136
4,23,ALB,150.68,1.08076,1327
5,31,AMD,92.46,1.02294,2163
6,179,FCX,26.63,0.998526,7510
7,85,CDNS,138.64,0.980741,1442
8,339,NOW,564.55,0.963938,354
9,236,IDXX,502.41,0.950181,398


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [33]:
hqm_cols = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Percentile',
    'Six-Month Price Return',
    'Six-Month Percentile',
    'Three-Month Price Return',
    'Three-Month Percentile',
    'One-Month Price Return',
    'One-Month Percentile'
]

hqm_df = pd.DataFrame(columns=hqm_cols)

for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch/stats?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_df = hqm_df.append(pd.Series(
            [symbol, 
            data[symbol]['price'], 
            'N/A',
            data[symbol]['stats']['year1ChangePercent'], 'N/A', 
            data[symbol]['stats']['month6ChangePercent'], 'N/A', 
            data[symbol]['stats']['month3ChangePercent'], 'N/A', 
            data[symbol]['stats']['month1ChangePercent'], 'N/A'], 
            index=hqm_cols), ignore_index=True)

hqm_df


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Percentile,Six-Month Price Return,Six-Month Percentile,Three-Month Price Return,Three-Month Percentile,One-Month Price Return,One-Month Percentile
0,A,124.40,N/A,0.409757,N/A,0.352903,N/A,0.180561,N/A,0.014258,N/A
1,AAL,16.00,N/A,-0.464505,N/A,0.20991,N/A,0.284546,N/A,0.116562,N/A
2,AAP,160.47,N/A,-0.0088786,N/A,0.114015,N/A,0.0281208,N/A,0.068358,N/A
3,AAPL,135.76,N/A,0.830891,N/A,0.47093,N/A,0.154682,N/A,0.11625,N/A
4,ABBV,109.68,N/A,0.278976,N/A,0.12041,N/A,0.250738,N/A,0.0252438,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.89,N/A,0.0990458,N/A,0.263694,N/A,0.196498,N/A,0.0268583,N/A
501,ZBH,155.05,N/A,0.0382469,N/A,0.304883,N/A,0.136673,N/A,0.0361937,N/A
502,ZBRA,397.86,N/A,0.523775,N/A,0.518121,N/A,0.548099,N/A,0.0159362,N/A
503,ZION,44.79,N/A,-0.132666,N/A,0.315339,N/A,0.52213,N/A,0.128617,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [40]:
time_periods = ['One-Year','Six-Month','Three-Month' ,'One-Month']
for i in range(len(hqm_df.index)):
    for period in time_periods:
        hqm_df.loc[i, f'{period} Percentile'] = stats.percentileofscore(hqm_df[f'{period} Price Return'], hqm_df.loc[i, f'{period} Price Return'])
hqm_df

TypeError: '<' not supported between instances of 'NoneType' and 'float'

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: